In [10]:
import pandas as pd
from fuzzywuzzy import process

pt_df = pd.read_csv("./data/pt_ratings.csv")
salaries_df = pd.read_csv("./data/salaries.csv")


In [11]:
pt_df

,courses,average_rating,type,reviews,name,slug
0,"['INST737', 'ENPM808W', 'ENPM808W', 'ENPM808W']",NaN,professor,[],A Seyed,seyed
1,"['MUSC800W', 'MUSC830W', 'MUSC830W']",NaN,professor,[],Aaron Goldman,goldman_aaron
2,"['THET678', 'THET499']",NaN,professor,[],Aaron Posner,posner
3,"['NFSC431', 'NFSC679R', 'NFSC112', 'HLTH672', ...",NaN,professor,[],Abani Pradhan,pradhan
4,"['CMLT235', 'ENGL101', 'ENGL234', 'ENGL101H', ...",NaN,professor,[],Abbey Morgan,morgan_abbey
...,...,...,...,...,...,...
11822,[],NaN,professor,[],Liangjun Shi,shi_liangjun
11823,[],NaN,professor,[],Rochelle Ford,ford_rochelle
11824,[],NaN,professor,[],Mark Stephen Graham,graham
11825,[],NaN,professor,[],Lena Morreale Scott,scott


In [14]:
assert len(pt_df) == len(pt_df.drop_duplicates(subset=["slug"]))

In [7]:
salaries_df["employee"] = salaries_df["employee"].str.replace("\n", " ")

In [15]:
for i, row in salaries_df.iterrows():
    parts = row["employee"].split(", ")
    # assert len(parts) == 2

    salaries_df.at[i, "name"] = ' '.join(parts[::-1])

pt_names = pt_df["name"].unique()

for name in salaries_df["name"].unique()[:10]:
    match = process.extract(name, pt_names)
    print(f"{name} -> {match}")


Eyad H Abed -> [('Eyad Abed', 95), ('Jerald Hage', 64), ('Gerald Hayes', 61), ('Gerald Haynes', 58), ('Masoud Hamedi', 58)]
Pamela A. Abshire -> [('Pamela Abshire', 95), ('A Seyed', 86), ('Pamela Orel', 86), ('A Anthony', 86), ('A Sharma', 86)]
Leigh R Abts -> [('Leigh Abts', 95), ('R Apter', 86), ('R Kelly', 86), ('Leigh Ryan', 73), ('Ashleigh Brown', 66)]
Douglas J Adams -> [('Douglas Hamilton (Astronomy)', 86), ('J Mihm', 86), ('J Perkins', 86), ('Joy Adams', 86), ('J Baum', 86)]
Vincent G. Adams -> [('Joy Adams', 86), ('G. Guttman', 86), ('Vincent Lee', 74), ('Vincent Bergamo', 71), ('Daniel Vincent', 66)]
Kofi Frimpong Addo -> [('Rong Ma', 64), ('Hong Kim', 64), ('Dong Dong', 64), ('Keong Kim', 61), ('Hong Jiao', 60)]
Raymond A. Adomaitis -> [('Raymond Adomaitis', 95), ('A Seyed', 86), ('Raymond Smith', 86), ('Raymond Weil', 86), ('A Kruglanski', 86)]
Mohamed Sherif Aggour -> [('Mohamed Aggour', 86), ('Mohamed Salem', 86), ('Elad Sherf', 72), ('Mohammed Charifa', 70), ('Amde Amde'